# compile posterior into a value-added catalog

In [ ]:
import os
import h5py
import fitsio
import numpy as np
from tqdm import tqdm
import astropy.table as aTable

In [2]:
from desitarget.sv3.sv3_targetmask import bgs_mask as sv3_bgs_mask

In [3]:
os.sys.path.append(os.getcwd().replace('/nb', '/bin'))
import svda as SVDA

In [4]:
import healpy


Bad key text.latex.preview in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95 

In [5]:
# target and redrcok outputs joined (see https://desi.lbl.gov/trac/wiki/ClusteringWG/LSScat/SV3/current_version)
#ff = fitsio.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/datcomb_bright_tarspecwdup_Alltiles.fits')
ff = fitsio.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/BGS_ANY_full.dat.fits')

In [6]:
# select only BGS targets
is_bgs_bright = (
    (ff['COADD_FIBERSTATUS'] == 0) & 
    ((ff['SV3_BGS_TARGET'] & sv3_bgs_mask['BGS_BRIGHT']) != 0) & 
    (ff['SPECTYPE'] == 'GALAXY') & 
    (ff['Z_HP'] > 0.) & (ff['Z_HP'] < 0.6) & 
    (ff['ZWARN'] == 0) & 
    (ff['DELTACHI2'] > 40.) & 
    (ff['ZERR'] < 0.0005 * (1 + ff['Z_HP'])))

is_bgs_faint = (
    (ff['COADD_FIBERSTATUS'] == 0) & 
    ((ff['SV3_BGS_TARGET'] & sv3_bgs_mask['BGS_FAINT']) != 0) & 
    (ff['SPECTYPE'] == 'GALAXY') & 
    (ff['Z_HP'] > 0.) & (ff['Z_HP'] < 0.6) & 
    (ff['ZWARN'] == 0) & 
    (ff['DELTACHI2'] > 40.) & 
    (ff['ZERR'] < 0.0005 * (1 + ff['Z_HP'])))

print('%i BGS BRIGHT z-success' % np.sum(is_bgs_bright))
print('%i BGS FAINT z-success' % np.sum(is_bgs_faint))

143074 BGS BRIGHT z-success
96771 BGS FAINT z-success


In [7]:
vagc = aTable.Table()

columns = ['TARGETID', 'RA', 'DEC', 'Z_HP', 'ZERR', 'TSNR2_BGS']
for col in columns: 
    vagc[col] = ff[col]

In [8]:
for col in ['MAG_G', 'MAG_R', 'MAG_Z', 'MAG_W1', 'FIBMAG_R', 'SCHLEGEL_COLOR']: 
    vagc[col] = np.repeat(-999., len(vagc))
vagc['HPIX_64'] = np.repeat(-999, len(vagc)).astype(int)

In [ ]:
vagc['is_bgs_bright'] = is_bgs_bright
vagc['is_bgs_faint'] = is_bgs_faint

In [9]:
for subsample in [is_bgs_bright, is_bgs_faint]: 

    photsys = ff['PHOTSYS'][subsample].astype(str)
    trans_g  = SVDA.mwdust_transmission(ff['EBV'][subsample], 'g', photsys, match_legacy_surveys=False)
    trans_r  = SVDA.mwdust_transmission(ff['EBV'][subsample], 'r', photsys, match_legacy_surveys=False)
    trans_z  = SVDA.mwdust_transmission(ff['EBV'][subsample], 'z', photsys, match_legacy_surveys=False)
    trans_w1 = SVDA.mwdust_transmission(ff['EBV'][subsample], 'z', photsys, match_legacy_surveys=False)

    flux_g  = ff['FLUX_G'][subsample] / trans_g
    flux_r  = ff['FLUX_R'][subsample] / trans_r
    flux_z  = ff['FLUX_Z'][subsample] / trans_z
    flux_w1 = ff['FLUX_W1'][subsample] / trans_w1
    fiberflux_r = ff['FIBERFLUX_R'][subsample] / trans_r

    vagc['MAG_G'][subsample]  = 22.5 - 2.5 * np.log10(flux_g.clip(1e-7))
    vagc['MAG_R'][subsample]  = 22.5 - 2.5 * np.log10(flux_r.clip(1e-7))
    vagc['MAG_Z'][subsample]  = 22.5 - 2.5 * np.log10(flux_z.clip(1e-7))
    vagc['MAG_W1'][subsample] = 22.5 - 2.5 * np.log10(flux_w1.clip(1e-7))

    vagc['FIBMAG_R'][subsample] = 22.5 - 2.5 * np.log10(fiberflux_r.clip(1e-7))
    
    phi = np.radians(vagc['RA'])[subsample]
    theta = np.radians(90. - vagc['DEC'])[subsample]
    vagc['HPIX_64'][subsample] = healpy.ang2pix(64, theta, phi, nest=True)
    
    vagc['SCHLEGEL_COLOR'][subsample] = ((vagc['MAG_Z'][subsample] - vagc['MAG_W1'][subsample]) 
                                         - 3./2.5 * (vagc['MAG_G'][subsample] - vagc['MAG_R'][subsample]) + 1.2)

In [10]:
uhpix = list(np.unique(vagc['HPIX_64']))
uhpix.remove(-999)

In [11]:
vagc['provabgs_mcmc'] = np.tile(-999., (len(vagc), 100, 13))
vagc['provabgs_theta_bf'] = np.tile(-999., (len(vagc), 13))
vagc['provabgs_z_max'] = np.tile(-999., len(vagc))
vagc['provabgs_logMstar_bf'] = np.tile(-999., len(vagc))
vagc['provabgs_logMstar'] = np.tile(-999., (len(vagc), 100))

In [13]:
dat_dir = '/global/cfs/cdirs/desi/users/chahah/provabgs/svda/'
for hpix in tqdm(uhpix): 
    for targ in ['BRIGHT', 'FAINT']: 
        fhpix = os.path.join(dat_dir, f'provabgs-sv3-bright-{hpix}.BGS_{targ}.hdf5')    
        if not os.path.isfile(fhpix): continue 
        #try: 
        with h5py.File(fhpix, 'r') as fhpix: 
            tids = fhpix['targetid'][...] # target id
            mcmc  = fhpix['samples'][...] # posterior samples
            mcmc  = mcmc.reshape((mcmc.shape[0], mcmc.shape[1] * mcmc.shape[2], mcmc.shape[3]))
            ttbf = fhpix['theta_bf'][...]

            zmax = fhpix['zmax'][...]
            logMs_bf = fhpix['logMstar_bf'][...]
            logMstar = fhpix['logMstar'][...]

        ngals = len(tids)
        for igal in range(ngals): 
            is_target = (vagc['TARGETID'] == tids[igal])
            if np.sum(is_target) > 1: 
                raise ValueError #print(np.sum(is_target), hpix, tids[igal])

            vagc['provabgs_mcmc'][is_target]     = mcmc[igal,-100:,:]
            vagc['provabgs_theta_bf'][is_target] = ttbf[igal]
            vagc['provabgs_z_max'][is_target]    = zmax[igal]

            vagc['provabgs_logMstar_bf'][is_target] = logMs_bf[igal]
            vagc['provabgs_logMstar'][is_target]    = logMstar[igal,-100:]

100%|██████████| 94/94 [00:00<00:00, 623.59it/s]


In [20]:
fout = '/global/cfs/cdirs/desi/users/chahah/provabgs/svda/BGS_ANY_full.provabgs.hdf5'
vagc.write(fout, overwrite=True)

/global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/astropy/io/misc/hdf5.py:283: UserWarning: table path was not set via the path= argument; using default path __astropy_table__
  "using default path {}".format(path))
